## 8.4 循环神经网络
在8.3节中，我们介绍了$n$元语法模型，其中单词$x_t$在时间步$t$的条件概率仅取决于前面$n-1$个单词。对于时间步$t-(n-1)$之前的单词，如果我们想将其可能产生的影响合并到$x_t$上，需要增加$n$，然而模型参数的数量也会随之呈指数增长，因为词表$V$需要存储$\mid V \mid ^n$个数字，因此与其将$P(x_t \mid x_{t-1}, ..., x_{t-n+1})$模型化，不如使用隐变量模型：
$$
P(x_t \mid x_{t-1}, ..., x_1) \approx P(x_t \mid h_{t-1})
$$
其中$h-1$是隐状态(hidden state)，也称为隐藏变量(hidden variable)，它存储了到时间步$t-1$的序列信息。通常，我们可以基于当前输入$x_t$和先前隐状态$h_{t-1}$来计算时间步t处的任何时间的隐状态：
$$
h_t = f(x_t, h_{t-1})
$$
对于(8.4.2)中的函数$f$，隐变量模型不是近似值。毕竟$h_t$是可以仅仅存储到目前为止观察到的所有数据，然而这样的操作可能会使计算的存储的代价都变得昂贵。

回想一下，我们在4节中讨论的具有隐藏单元的隐藏层。值得注意的是，隐藏层和隐状态指的是两个截然不同的概念。如上所述，隐藏层是在从输入到输出的路径上(以观测角度来理解)的隐藏的层，而隐状态则是在给定步骤所做的任何事情(以技术角度来定义)的输入，并且这些状态只能通过先前时间步的数据来计算。

循环神经网络(recurrent neural networks, RNNs)是具有隐状态的神经网络。在介绍循环神经网络模型之前，我们先回顾一下多层感知机模型。

### 8.4.1 无隐状态的神经网络
让我们来看一看只有单隐藏层的多层感知机。设隐藏层的激活函数为$\phi$，给定一个小批量样本$X \in \mathbb R^{n \times d}$，其中批量大小为n，输入维度为d，则隐藏层的输出$H \in \mathbb R^{n \times h}$通过下式计算：
$$
H = \phi (XW_{xh} + b_h)
$$
在(8.4.3)中，我们拥有的隐藏层权重参数为$W_{xh} \in \mathbb R^{d \times h}$，偏置参数为$b_h \in \mathbb R^{1 \times h}$，以及隐藏单元的数目为h。因此求和时可以应用广播机制。接下来，将隐藏变量$H$用作输出层的输入。输出层由下式给出：
$$
O = HW_{hq} + b_q
$$

其中，$O \in \mathbb R^{n \times q}$是输出变量，$W_{hq} \in \mathbb R^{h \times q}$是权重参数，$b_q \in \mathbb R^{1 \times q}$是输出层的偏置参数。如果是分类问题，我们可以用$softmax(O)$来计算输出类别的概率分布。

这完全类似于之前在8.1节中解决的回归问题，因此我们省略了细节。无需多言，只要可以随机选择“特征-标签”对，并且通过自动微分和随机梯度下降能够学习网络参数就可以了。


### 8.4.2 有隐状态的循环神经网络
有了隐状态后，情况就完全不同了。假设我们在时间步t有小批量输入$X_t \in \mathbb R^{n \times d}$。换言之，对于n个序列样本的小批量，$X_t$的每一行对应于来自该序列的时间步t处的一个样本。接下来，用$H_t \in \mathbb R^{n \times h}$表示时间步t的隐藏变量。与多层感知机不同的是，我们在这里保存了前一个时间步的隐藏变量$H_{t-1}$，并引入了一个新的权重参数$W_hh \in \mathbb R^{h \times h}$，来描述如何在当前时间步的隐藏变量。具体地说，当前时间步隐藏变量由当前时间步的输入与前一个时间步的隐藏变量一起计算得出：
$$
H_t = \phi (X_t W_{xh} + H_{t-1} W_{hh} + b_h)
$$

与(8.4.3)相比，(8.4.5)多了一项$H_{t-1}W_{hh}$，从而实例化了(8.4.2)。从相邻时间步的隐藏变量$H_t$和$H_{t-1}$之间的关系可知，这些变量捕获并保留了序列直到其当前时间步的历史信息，就如当前时间步下神经网络的状态或记忆，因此这样的隐藏变量被称为隐状态(hidden state)。由于在当前时间步中，隐状态使用的定义与前一个时间步中使用的定义相同，因此(8.4.5)的计算是循环的(recurrent)。于是基于循环计算的隐状态神经网络被命名为循环神经网络(recurrent neural network)。在循环神经网络中执行(8.4.5)计算的层称为循环层(recurrent layer)。


有许多不同的方法可以构建循环神经网络，由(8.4.5)定义的隐状态的循环神经网络时非常常见的一种。对于时间步t，输出层的输出类似于多层感知机的计算：
$$
O_t = H_t W_{hq} + b_q
$$
循环神经网络的参数包括隐藏层的权重$W_{xh} \in \mathbb R^{d \times h}, W_{hh} \in \mathbb R^{h \times h}$和偏执$b_h \in \mathbb R^{1 \times h}$，以及输出层的权重$W_{hq} \in \mathbb R^{h \times q}$和偏置$b_{q} \in \mathbb R^{1 \times q}$。值得一提的是，即使在不同的时间步，循环神经网络也总是使用这些模型参数。因此，循环神经网络的参数开销不会随着时间步的增加而增加。

图8.4.1展示了循环神经网络在三个相邻时间步的计算逻辑。在任意时间步t，隐状态的计算可以被视为：
1. 拼接当前时间步t的输入$X_t$和前一时间步$t -1 $的隐状态$H_{t-1}$；
2. 将拼接的结果送入带有激活函数$\phi$的全连接层。全连接层的输出是当前时间步t的隐状态$H_t$

在本例，模型参数是$W_{xh}$和$W_{hh}$的拼接，以及$b_h$的偏置，所有这些参数都来自(8.4.5)。当前时间步t的隐状态$H_t$将参与计算下一时间步$t+1$的隐状态$H_{t+1}$。而且$H_t$还将送入全连接输出层，用于计算当前时间步t的输出$O_t$。

<div align=center>
<img src='../../pics/8_4_1.jpeg' width='50%'>
</div>


我们刚提到，隐状态$X_t W_{xh} + H_{t-1} W_{hh}$的计算，相当于$X_t$和$H_{t-1}$的拼接与$W_{xh}$和$W_{hh}$的拼接的矩阵乘法。虽然这个性质可以通过数学证明，但在下面我们使用一个简单的代码来说明一下。首先，我们定义矩阵X、W_xh、 H和W_hh，它们的形状分别为(3 \otimes 1)、(1 \otimes 4)、(3 \otimes 4)和(4 \otimes 4)。分别将X乘以W_xh，将H乘以W_hh，然后将这两个乘法想加，我们得到一个形状为(3 \otimes 4)的矩阵。

In [3]:
import torch
from d2l import torch as d2l

X, W_xh = torch.normal(0, 1, (3, 1)), torch.normal(0, 1, (1, 4))
H, W_hh = torch.normal(0, 1, (3, 4)), torch.normal(0, 1, (4, 4))

torch.matmul(X, W_xh) + torch.matmul(H, W_hh)

tensor([[ 0.7295, -0.8228,  0.8028, -3.2281],
        [ 1.8390,  1.4227, -0.7348,  1.9641],
        [ 0.1396, -0.7171, -1.4005,  2.4501]])

现在，我们沿列(轴1)拼接矩阵X和H，沿行(轴0)拼接矩阵W_xh和W_hh。这两个拼接分别阐述形状(3,5)和形状(5, 4)的矩阵。再将这两个拼接的矩阵相乘，我们得到与上面相同形状(3, 4)的输出矩阵

In [4]:
torch.matmul(torch.cat((X, H), 1), torch.cat((W_xh, W_hh), 0))

tensor([[ 0.7295, -0.8228,  0.8028, -3.2281],
        [ 1.8390,  1.4227, -0.7348,  1.9641],
        [ 0.1396, -0.7171, -1.4005,  2.4501]])

### 8.4.3 基于循环神经网络的字符级语言模型
回想一下8.3节的语言模型，我们的目标是根据过去的和当前的词元预测下一个词元，因此我们将原始序列移位一个词元作为标签。Bengio等人首先提出使用神经网络进行语言建模。接下来，我们看一下如何使用循环神经网络来构建语言模型。设小批量大小为1，批量中的那个文本序列为“machine”。为了简化后续部分的训练，我们考虑使用字符级语言模型(character-level language model)，将文本词元化为字符而不是单词。图8.4.2演示了如何通过基于字符级语言建模的循环神经网络，使用当前的和先前的字符预测下一个字符。

<div align=center>
<img src='../../pics/8_4_2.jpeg' width="50%">
</div>

在训练过程中，我们对每个时间步的输出层的输出进行softmax操作，然后利用交叉熵损失计算模型输出和标签之间的误差。由于隐藏层中隐状态的循环计算，图8.4.2中的第3个时间步的输出$O_3$由文本序列“m”、“a”和“c”确定。由于训练数据中这个文本序列的下一个字符是“h”，因此第3个时间步的损失将取决于下一个字符的概率分布，而下一个字符是基于特征序列“m”、“a”、“c”和这个时间步的标签“h”生成的。

在实践中，我们使用的批量大小为$n>1$，每个词元都由一个d维向量表示。因此，在时间步t输入$X_t$将是一个$n \times d$矩阵，这与我们在8.4.2节中的讨论相同。


### 8.4.4 困惑度